In [1]:
from agents import *

In [44]:
class MAgent(Agent):
    def __init__(self, program=None):
        super().__init__(program)
        self.things = []
        self.last_action = "none"
        self.sign = "a "
        
    # Movements
    
    def moveRight(self):
        self.location = (self.location[0] + 1, self.location[1])

    def moveLeft(self):
        self.location = (self.location[0] - 1, self.location[1])

    def moveDown(self):
        self.location = (self.location[0], self.location[1] + 1)

    def moveUp(self):
        self.location = (self.location[0], self.location[1] - 1)

    def noOp(self):
        pass
    
    # Agent State

    def add_perecpted_things(self, things):
        for thing in things:
            self.things.append(thing)

    def thing_at(self, location):
        for thing in self.things:
            if thing.location == location:
                return thing.sign
        return "? "

    def show_state(self):
        print('\n' + " Agent's internal state: " + '\n')
        print("  0 1 2 3 4 5 ")
        for y in range(6):
            print(y, end=' ')
            for x in range(6):
                print(self.thing_at((x, y)), end='')
            print()

In [57]:
class AThing(Thing):
    def __init__(self,value,sign,location=(0,0)):
        self.location = location
        self.value = value
        self.sign = sign

class Nothing(AThing):
    def __init__(self,location=(0,0)):
        super().__init__(-1,"- ",location)

class Outside(AThing):
    def __init__(self,location=(0,0)):
        super().__init__(-10,"o ",location)

class Wall(AThing):
    def __init__(self):
        super().__init__(-5,"w ")

class Treasure_gold(AThing):
    def __init__(self):
        super().__init__(20,"g ")

class Treasure_diamond(AThing):
    def __init__(self):
        super().__init__(50,"d ")

class Snake_trap(AThing):
    def __init__(self,):
        super().__init__(-20,"s ")

class Hole_trap(AThing):
    def __init__(self):
        super().__init__(-15,"h ")

In [22]:
def remove_invalid(percepts):
    percs = []
    for x in percepts:
        if isinstance(x,Outside) or isinstance(x,Wall):
            pass
        else:
            percs.append(x)
    return percs

In [80]:
class MEnv(Environment):    
        
    # Things displayed (different from thing_at, because this includes the agent)
    def disp_thing_at(self,agent,location):
        if agent.location == location:
            return agent
        elif location[0] < 0 or location[1] < 0 or location[0] > 5 or location[1] > 5:
            return Outside(location)
        elif self.list_things_at(location):
            return self.list_things_at(location)[0]
        else:
            return Nothing(location)
        
    # Different from disp_thing_at, because this excludes the agent)    
    def thing_at(self, agent, location):
        if location[0] < 0 or location[1] < 0 or location[0] > 5 or location[1] > 5:
            return Outside(location)
        elif self.list_things_at(location):
            # This returns the thing that isn't the agent it doesn't found anything returns nothing
            if agent in self.list_things_at(location):
                for t in self.list_things_at(location):
                    if not isinstance(t,MAgent):
                        return t
                return Nothing(location)
            else:
                return self.list_things_at(location)[0]
        else:
            return Nothing(location)

    def percept(self, agent):
        location = agent.location

        here = self.thing_at(agent, location)
        left = self.thing_at(agent, (location[0] - 1, location[1]))  # Left
        down = self.thing_at(agent, (location[0], location[1] + 1))  # Down
        right = self.thing_at(agent, (location[0] + 1, location[1]))  # Right
        up = self.thing_at(agent, (location[0], location[1] - 1))  # Up

        here.position = "nop"
        left.position = "left"
        down.position = "down"
        right.position = "right"
        up.position = "up"

        percepted_things = [here, left, down, up, right]
        
        print("<BEFORE THE ACTION>")
        self.show_grid(agent)
        agent.add_perecpted_things(percepted_things)
        agent.show_state()
        print("Agent performance: ",agent.performance)
        print("<AGENT PROGRAM>")
        print("Percept: ({},({},{},{},{},{}))".format(location, percepted_things[0].sign, percepted_things[1].sign, percepted_things[2].sign, percepted_things[3].sign, percepted_things[4].sign))
        return {"location": location, "positions": remove_invalid(percepted_things) } #agent.things


    def execute_action(self, agent, action):
        print("Action: ",action)
        if action == "nop":
            pass
        else:
            agent.performance -= 1
            if action == "right":
                agent.moveRight()
            elif action == "left":
                agent.moveLeft()
            elif action == "up":
                agent.moveUp()
            elif action == "down":
                agent.moveDown()
            elif action == "nop":
                agent.noOp

        location = agent.location
        thing = self.thing_at(agent,location)
        
        # Discriminate path already covered to prevent loops
        if isinstance(thing,Nothing):
            self.add_thing(Nothing(),location)
        thing.value = thing.value - 1
        
        if isinstance(thing, Treasure_gold) or isinstance(thing, Treasure_diamond):
            self.delete_thing(thing)
            agent.performance += thing.value

        if isinstance(thing, Snake_trap) or isinstance(thing, Hole_trap):
            agent.performance += thing.value
            
        agent.last_action = action
        print("<AFTER THE ACTION>")
        self.show_grid(agent)
        agent.show_state()
        print("Agent performance: ",agent.performance)
        print("----------------------------------------------------------------------")

    def show_grid(self,agent):
        print(" The environment content: " + '\n')
        print("  0 1 2 3 4 5 ")
        for y in range(6):
            print(y, end=' ')
            for x in range(6):
                print(self.disp_thing_at(agent,(x, y)).sign, end='')
            print()

In [81]:
def areEquals(e):
    if e[0] == e[1] == e[2] == e[3]:
        return True
    else:
        return False

def program(percepts):
    
    if areEquals(percepts["positions"]):
        best_choice = random.choice(percepts["positions"])
        return best_choice
    else:
        xx = max(percepts["positions"], key=lambda x: x.value).position
        best_choice = xx
        return best_choice

In [1]:
e = MEnv()

# Isra added stuff
w = Wall()
g = Treasure_gold()
d = Treasure_diamond()
s = Snake_trap()
h = Hole_trap()

#row 0
e.add_thing(Treasure_gold(), (0, 2))
#row 1
e.add_thing(Hole_trap(), (0, 1))
e.add_thing(Snake_trap(), (2, 1))
e.add_thing(Wall(), (3, 1))
#row 2
e.add_thing(Treasure_diamond(), (0, 3))
e.add_thing(Wall(), (3, 2))
e.add_thing(Treasure_diamond(), (5, 2))
#row 3
#e.add_thing(Wall(),(3,3))
e.add_thing(Wall(), (2, 3))
e.add_thing(Wall(), (1, 3))
#row 4
e.add_thing(Hole_trap(), (3, 4))
e.add_thing(Treasure_diamond(), (4, 4))
#row 5
e.add_thing(Snake_trap(), (0, 5))

# Isra ended adding stuff



a = MAgent(program)
#TraceAgent(a)
e.add_thing(a, (0, 2))
e.run(100)

NameError: name 'MEnv' is not defined